# RAG 2

In [4]:
import openai
import numpy as np
import inspect
from functools import lru_cache
import requests
from bs4 import BeautifulSoup

In [182]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

import openai
client = openai.OpenAI()

In [8]:
# Cache para las respuestas
_cache = {}

# Función para crear claves de caché
def make_cache_key(kwargs):
    def convert_to_hashable(value):
        if isinstance(value, dict):
            return frozenset((k, convert_to_hashable(v)) for k, v in sorted(value.items()))
        elif isinstance(value, list):
            return tuple(convert_to_hashable(v) for v in value)
        elif isinstance(value, set):
            return frozenset(convert_to_hashable(v) for v in value)
        else:
            return value

    return frozenset((k, convert_to_hashable(v)) for k, v in sorted(kwargs.items()))

# Función que interactúa con la API de OpenAI y utiliza la caché
def llm(**kwargs):
    key = make_cache_key(kwargs)
    try:
        return _cache[key]
    except KeyError:
        _cache[key] = result = client.chat.completions.create(**kwargs)
        return result

# Decorador para la caché LRU
@lru_cache(3000)
def get_embedding(text, model="text-embedding-3-large"):
    client = openai.OpenAI()
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Función para obtener funciones de un módulo
def get_module_functions(module_name):
    module = __import__(module_name)
    functions = []

    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj) or inspect.isbuiltin(obj):
            ds = inspect.getdoc(obj)
            function_info = {
                "name": name,
                "docstring": ds
            }
            if ds:
                functions.append(function_info)

    return functions

In [119]:
# Función para generar resúmenes utilizando OpenAI
def generate_summary(text, max_tokens=3000):
    prompt = f"Resumir el siguiente texto:\n{text}\n\nResumen:"
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",  # Selecciona el modelo adecuado
        prompt=prompt,
        max_tokens=max_tokens
    )
    summary = response.choices[0].text.strip()
    return summary

# Ejemplo de uso de la función
text_to_summarize = "Este es un largo texto que queremos resumir utilizando OpenAI. Podemos usar este texto como ejemplo para ver cómo funciona la generación de resúmenes."
summary = generate_summary(text_to_summarize)
print("Resumen generado por OpenAI:", summary)


Resumen generado por OpenAI: El texto es utilizado como ejemplo para mostrar cómo funciona la generación de resúmenes utilizando OpenAI.


In [10]:
# Función para calcular la similitud coseno
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Función para consultar la base de datos
def query_bbdd(query, top_n=5):
    query_embedding = get_embedding(query)
    
    # Calcular la similitud coseno entre la consulta y cada embedding de los chunks
    similarities = [cosine_similarity(query_embedding, embedding) for embedding in embeddings]
    
    # Obtener los índices de los top-n chunks más similares
    top_indices = np.argsort(similarities)[-top_n:]
    
    # Recuperar los top-n chunks más similares junto con su similitud coseno
    top_chunks = [(names[i], chunks[i], similarities[i]) for i in top_indices]
    
    return top_chunks

In [14]:
# Ejemplos predefinidos (pueden ser preguntas frecuentes o ejemplos de código)
# examples = [
#     {"question": "¿Cómo defino una función en Python?", "answer": "Puedes definir una función en Python usando la palabra clave 'def' seguida del nombre de la función y paréntesis. Por ejemplo:\n\n```python\ndef mi_funcion():\n    print('Hola Mundo')\n```"},
#     {"question": "¿Cómo uso listas en Python?", "answer": "Las listas en Python se definen usando corchetes. Puedes añadir, eliminar y acceder a elementos en una lista. Por ejemplo:\n\n```python\nmi_lista = [1, 2, 3]\nmi_lista.append(4)\nprint(mi_lista)  # Salida: [1, 2, 3, 4]\n```"},
#    # Añade más ejemplos según sea necesario
# ]

# Extraer nombres y embeddings de los ejemplos
# names = [example["question"] for example in examples]
# chunks = [example["answer"] for example in examples]
# embeddings = [get_embedding(name) for name in names]

# Función principal del chatbot
# def chatbot(query):
#    # Buscar en la base de datos para encontrar ejemplos relevantes
#    results = query_bbdd(query)
    
    # Generar la respuesta basada en los ejemplos encontrados
#    response = ""
#    for name, chunk, similarity in results:
#        response += f"Pregunta: {name}\nRespuesta: {chunk}\n\n"

#    return response

# Ejemplo de uso del chatbot
# query = "¿Cómo defino una función en Python?"
# response = chatbot(query)
# print(response)

In [135]:
# Función para extraer contenido de una URL
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    summary = generate_summary(text[:3000], max_tokens=300)
    # paragraphs = soup.find_all('p')
    # return ' '.join([para.get_text() for para in paragraphs])
    return summary

In [131]:
# URLs de páginas específicas de Python.org
# urls = {
#    "Comenzar con Python":"https://docs.python.org/3/tutorial/appetite.html",
#    "Utilizando el Interprete":"https://docs.python.org/3/tutorial/interpreter.html",
#    "Introducción":"https://docs.python.org/3/tutorial/introduction.html",
#    "Control de Flujo":"https://docs.python.org/3/tutorial/controlflow.html",
#    "Estructuras de Datos":"https://docs.python.org/3/tutorial/datastructures.html",
#    "Modulos":"https://docs.python.org/3/tutorial/modules.html",
#    "Entrada y Salida":"https://docs.python.org/3/tutorial/modules.html",
#    "Errores y Excepciones":"https://docs.python.org/3/tutorial/errors.html",
#    "Clases":"https://docs.python.org/3/tutorial/classes.html",
#    "Librería Estandar":"https://docs.python.org/3/tutorial/stdlib.html",
#    "Más Librería Estandar":"https://docs.python.org/3/tutorial/stdlib2.html",
#    "Entornos virtuales y Paquetes":"https://docs.python.org/3/tutorial/stdlib2.html",
#    "Más Información":"https://docs.python.org/3/tutorial/whatnow.html",
#    "Interprete Interactivo":"https://docs.python.org/3/tutorial/interactive.html",
#    "Limitaciones de Coma Flotante":"https://docs.python.org/3/tutorial/floatingpoint.html",
#    "Apendice":"https://docs.python.org/3/tutorial/appendix.html"
    # Añade más URLs relevantes
# }

urls = {
    "Definir una Función": "https://docs.python.org/3/tutorial/controlflow.html#defining-functions",
    "Listas en Python": "https://docs.python.org/3/tutorial/introduction.html#lists",
    "Tuplas y Secuencias": "https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences",
    "Diccionarios en Python": "https://docs.python.org/3/tutorial/datastructures.html#dictionaries",
    "Sets": "https://docs.python.org/3/tutorial/datastructures.html#sets",
    "Estructuras de Control de Flujo": "https://docs.python.org/3/tutorial/controlflow.html",
    "Sentencias Condicionales": "https://docs.python.org/3/tutorial/controlflow.html#if-statements",
    "Bucles While": "https://docs.python.org/3/tutorial/introduction.html#first-steps-towards-programming",
    "Bucles For": "https://docs.python.org/3/tutorial/controlflow.html#for-statements",
    "Funciones Lambda": "https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions",
    "Manejo de Excepciones": "https://docs.python.org/3/tutorial/errors.html",
    "Clases y Objetos": "https://docs.python.org/3/tutorial/classes.html",
    "Herencia en Python": "https://docs.python.org/3/tutorial/classes.html#inheritance",
    "Módulos en Python": "https://docs.python.org/3/tutorial/modules.html",
    "Entrada y Salida de Archivos": "https://docs.python.org/3/tutorial/inputoutput.html",
    "Manejo de Argumentos de Línea de Comandos": "https://docs.python.org/3/library/argparse.html",
    "Expresiones Regulares": "https://docs.python.org/3/library/re.html",
    "Gestión de Fechas y Horas": "https://docs.python.org/3/library/datetime.html",
    "Operaciones de Sistema": "https://docs.python.org/3/library/os.html",
    "Interactuando con el Sistema Operativo": "https://docs.python.org/3/library/os.path.html"
}

In [137]:
# Extraer el contenido de las URLs y calcular sus embeddings
names = list(urls.keys())
chunks = [extract_text_from_url(url) for url in urls.values()]
embeddings = [get_embedding(chunk) for chunk in chunks]

In [31]:
PROMPT_TEMPLATE = f"""
Eres un chatbot experto en la biblioteca estándar de Python. 

Debes ayudar a explicar código Python, comentar código Python, corregir código Python o desarrollar código Python. 
NO debes contestar ninguna pregunta NO relacionada con Python. Si te hacen alguna pregunta no relacionada con Python DEBES rechazar responder amablemente.

Utiliza la siguiente base de conocimientos para responder a la pregunta del usuario:

Base de conocimientos: {context}

---

Usuario: {question}
"""

In [33]:
# Función principal del chatbot
def chatbot(query):
    # Buscar en la base de datos para encontrar ejemplos relevantes
    results = query_bbdd(query)
    
    # Generar la respuesta basada en los ejemplos encontrados
    response = ""
    for name, chunk, similarity in results:
        response += f"Pregunta: {name}\nRespuesta: {chunk}\n\n"

    return response

In [ ]:
query_bbdd("¿Cómo defino una función en Python?")

In [178]:
def chatbot_rag(pregunta):

    results = query_bbdd(pregunta)
    if len(results) == 0:
        print(f"Unable to find matching results.")
        return

    context = "\n\n---\n\n".join([doc for _name, doc, _score in results])
    messages=[
        {"role": "system", "content": f"""
Eres un chatbot experto en la biblioteca estándar de Python. 

Debes ayudar a explicar código Python, comentar código Python, corregir código Python o desarrollar código Python. 
NO debes contestar ninguna pregunta NO relacionada con Python. 
Si te hacen alguna pregunta no relacionada con Python DEBES rechazar responder amablemente y hacer un chiste.


Utiliza la siguiente base de conocimientos para responder a la pregunta del usuario:

Base de conocimientos: {context}

"""},
        {"role": "user", "content": pregunta}
]
    # print(prompt)

    response_text = llm(max_tokens=3000, model="gpt-3.5-turbo-1106",
                                       messages=messages)


    # print(response_text)
    return response_text

In [180]:
# Ejemplo de uso del chatbot
query = "¿Cómo se hace una tortilla de patatas?"
response = chatbot_rag(query)
print(response.choices[0].message.content)

¡Claro, te puedo ayudar a preparar una "tortilla de patatas" en Python! Primero, necesitaríamos importar el módulo "cocina" y luego usar la función "preparar_tortilla" para crear la receta. ¡Ups! Parece que me equivoqué, ¡tú querías una receta de cocina, no de programación! 😄


In [167]:
print(response.choices[0].message.content)

Para convertir un diccionario en una lista en Python, puedes utilizar la función `list()` aplicada al diccionario. Esto convertirá las claves del diccionario en elementos de la lista. Aquí tienes un ejemplo:

```python
mi_diccionario = {'a': 1, 'b': 2, 'c': 3}
mi_lista = list(mi_diccionario)
print(mi_lista)  # Salida: ['a', 'b', 'c']
```

En este caso, la función `list()` convierte las claves del diccionario en una lista de strings. Si quieres convertir los valores del diccionario en una lista, puedes usar el método `values()` del diccionario, de la siguiente forma:

```python
valores = list(mi_diccionario.values())
print(valores)  # Salida: [1, 2, 3]
```

Espero que esta información te sea útil. Si tienes alguna otra pregunta, no dudes en preguntar.
